In [6]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from DDPG import DDPGagent
from utils_aaa import *
import traci
import traci.constants as tc
import math
import torch
import torch.autograd
import torch.optim as optim
import torch.nn as nn
from collections import deque
import random
from model_01 import *
from utils_aa import *
from memory import *

In [7]:
import os, sys
tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
sys.path.append(tools)

In [10]:
import random
random.seed()
r=36.61
rl1='autonomous_vehicle'
num_veh=42
speed_total=np.zeros(42)
pos_total=np.zeros(42)
state_total_01=np.zeros(10)
action_total_01=np.zeros(2)
this_lane=0
for episode in range(1):
#    state = env.reset()
#traci.start(["sumo-gui", "-c", "/Users/yuewang/document/maps280/ring_example_20200319-1819061584627546.990043.sumo.cfg"])
    traci.start(["sumo-gui", "-c", "/Users/yuewang/document/maps_multi-lane-ring/ring_example_20190912-1246031568277963.025501.sumo.cfg"])
    traci.simulationStep()
    vehID=traci.vehicle.getIDList()

    depart=None
    speed0="0"
    lane="0"
    typeID="human"
    routeID="routebottom"
    for i in range(7):
        str1=["v",str(i),"_0"]
        vehID="".join(str1)
        departPos_e=max(8.0*((i)),0)
        departPos=str(departPos_e)
        traci.vehicle.add(vehID, routeID, typeID, depart, lane, departPos, speed0)

    routeID="routeright"
    for i in range(7,14):
        str1=["v",str(i),"_0"]
        vehID="".join(str1)
        departPos_e=max(8.0*((i-7)),0)
        departPos=str(departPos_e)
        traci.vehicle.add(vehID, routeID, typeID, depart, lane, departPos, speed0)

    routeID="routetop"
    for i in range(14,21):
        str1=["v",str(i),"_0"]
        vehID="".join(str1)
        departPos_e=max(8.0*((i-14)),0)
        departPos=str(departPos_e)
        traci.vehicle.add(vehID, routeID, typeID, depart, lane, departPos, speed0)

    lane="1"
    typeID="human"
    routeID="routebottom"
    for i in range(7):
        str1=["v",str(i),"_1"]
        vehID="".join(str1)
        departPos_e=max(8.0*((i)),0)
        departPos=str(departPos_e)
        traci.vehicle.add(vehID, routeID, typeID, depart, lane, departPos, speed0)

    routeID="routeright"
    for i in range(7,14):
        str1=["v",str(i),"_1"]
        vehID="".join(str1)
        departPos_e=max(8.0*((i-7)),0)
        departPos=str(departPos_e)
        traci.vehicle.add(vehID, routeID, typeID, depart, lane, departPos, speed0)

    routeID="routetop"
    for i in range(14,20):
        str1=["v",str(i),"_1"]
        vehID="".join(str1)
        departPos_e=max(8.0*((i-14)),0)
        departPos=str(departPos_e)
        traci.vehicle.add(vehID, routeID, typeID, depart, lane, departPos, speed0)

    typeID="rl1"
    departPos_e=max(9.0*((20-14)),0)
    departPos=str(departPos_e)
    traci.vehicle.add("autonomous_vehicle", routeID, typeID, depart, lane, departPos, speed0)
    traci.simulationStep()
    vehID=traci.vehicle.getIDList()

    for t in range(len(vehID)):
        traci.vehicle.setSpeedMode(vehID[t],31)
        traci.vehicle.setLaneChangeMode(vehID[t],512)



# reset state
    for reset_state in range(1):
        traci.simulationStep()
        for j in range(len(vehID)):
            testVname=vehID[j]
 
            vehposition=traci.vehicle.getPosition(testVname)
            vehpos_1=vehposition[0]-r
            vehpos_2=vehposition[1]-r
        
            if vehpos_1>0 and vehpos_2<0:
                traci.vehicle.setRouteID(testVname, 'routebottom')
    
            if vehpos_1>0 and vehpos_2>0:
                traci.vehicle.setRouteID(testVname, 'routeright')
    
            if vehpos_1<0 and vehpos_2>0:
                traci.vehicle.setRouteID(testVname, 'routetop')
    
            if vehpos_1<0 and vehpos_2<0:
                traci.vehicle.setRouteID(testVname, 'routeleft')

            traci.vehicle.subscribe(testVname,(tc.VAR_LANE_INDEX,tc.VAR_SPEED,tc.VAR_POSITION))
    

    vehicle_obs_rl1=traci.vehicle.getSubscriptionResults(vehID[0])
    lane_rl1=vehicle_obs_rl1[tc.VAR_LANE_INDEX]
    veh_len=len(vehID)
    #set the line
    vehposition=vehicle_obs_rl1[tc.VAR_POSITION]
    vehpos_1=vehposition[0]-r
    vehpos_2=vehposition[1]-r
    
    slope=vehpos_2/vehpos_1

    theta_total=1000*np.ones(veh_len)
    theta_total_01=1000*np.ones(veh_len)

    for i in range(1,len(vehID)):
        vehicle_obs_e=traci.vehicle.getSubscriptionResults(vehID[i])
        lane_e=vehicle_obs_e[tc.VAR_LANE_INDEX]
        vehposition_e=vehicle_obs_e[tc.VAR_POSITION]
        vehpos_1_e=vehposition_e[0]-r
        vehpos_2_e=vehposition_e[1]-r
        ind_e=vehpos_2_e-slope*vehpos_1_e
    
        if slope>0 and vehpos_1 !=0 and vehpos_1>0:
            if ind_e<=0 and lane_e != lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total[i]=theta
            if ind_e<=0 and lane_e == lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total_01[i]=theta
 
    
        if slope>0 and vehpos_1 !=0 and vehpos_1<0:
            if ind_e>=0 and lane_e != lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total[i]=theta
            if ind_e<=0 and lane_e == lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total_01[i]=theta
    
        if slope<0 and vehpos_1 !=0 and vehpos_1>0:
            if ind_e<=0 and lane_e != lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total[i]=theta
            if ind_e<=0 and lane_e == lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total_01[i]=theta                

    
        if slope<0 and vehpos_1 !=0 and vehpos_1<0:
            if ind_e>=0 and lane_e != lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total[i]=theta
            if ind_e<=0 and lane_e == lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total_01[i]=theta
                
    
        if slope==0 and vehpos_1>0:
            if vehpos_2_e<=0 and lane_e != lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total[i]=theta
            if ind_e<=0 and lane_e == lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total_01[i]=theta
                
    
        if slope==0 and vehpos_1<0:
            if vehpos_2_e>=0 and lane_e != lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total[i]=theta
            if ind_e<=0 and lane_e == lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total_01[i]=theta
    
        if vehpos_1==0 and vehpos_2>0:
            if vehpos_1_e>=0 and lane_e != lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total[i]=theta
            if ind_e<=0 and lane_e == lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total_01[i]=theta
                
    
        if vehpos_1==0 and vehpos_2<0:
            if vehpos_1_e<=0 and lane_e != lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total[i]=theta
            if ind_e<=0 and lane_e == lane_rl1:
                cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                theta=math.acos(cos)
                theta_total_01[i]=theta


    y=np.where(theta_total==min(theta_total))
    veh_nextBehind=vehID[int(y[0])]
    vehicle_obs_e=traci.vehicle.getSubscriptionResults(veh_nextBehind)
    speed_nextBehind=vehicle_obs_e[tc.VAR_SPEED]
    
    y_01=np.where(theta_total_01==min(theta_total_01))
    veh_Behind=vehID[int(y_01[0])]
    vehicle_obs_e=traci.vehicle.getSubscriptionResults(veh_Behind)
    speed_Behind=vehicle_obs_e[tc.VAR_SPEED]
    dist_Behind=(min(theta_total_01))*r

    headway=traci.vehicle.getLeader(veh_nextBehind,200)
    headway1=headway[0]
    headway2=headway[1]

    dist=headway2
    header_rl1=headway1

    vehicle_header=traci.vehicle.getSubscriptionResults(header_rl1)
    veh_header=vehicle_header[tc.VAR_SPEED]

    veh=vehicle_obs_rl1[tc.VAR_SPEED]
    currentLaneIndex=lane_rl1

    diff_dist=(min(theta_total))*r
    headway_nextBehind=dist
    speed_nextLeader=veh_header

    state=np.array([veh,veh_header,dist,currentLaneIndex,speed_Behind,dist_Behind,speed_nextBehind,speed_nextLeader,headway_nextBehind,diff_dist])

    last_lc=0
    
#start training
    for step in range(4000):      
        
        state_01 = Variable(torch.from_numpy(state).float().unsqueeze(0))
        state_total_01=np.vstack((state_total_01,state))
        
        traci.simulationStep()
# done
        pos_e=np.zeros(42)
        for j in range(len(vehID)):
            testVname=vehID[j]
 
            vehposition=traci.vehicle.getPosition(testVname)
                
            vehpos_1=vehposition[0]-r
            vehpos_2=vehposition[1]-r
            if vehpos_1>0 and vehpos_2<0:
                traci.vehicle.setRouteID(testVname, 'routebottom')
                dist_pos_e=traci.vehicle.getLanePosition(testVname)
                dist_pos=dist_pos_e
            if vehpos_1>0 and vehpos_2>0:
                traci.vehicle.setRouteID(testVname, 'routeright')
                dist_pos_e=traci.vehicle.getLanePosition(testVname)
                dist_pos=dist_pos_e+230/4
            if vehpos_1<0 and vehpos_2>0:
                traci.vehicle.setRouteID(testVname, 'routetop')
                dist_pos_e=traci.vehicle.getLanePosition(testVname)
                dist_pos=dist_pos_e+230/4*2
            if vehpos_1<0 and vehpos_2<0:
                traci.vehicle.setRouteID(testVname, 'routeleft')
                dist_pos_e=traci.vehicle.getLanePosition(testVname)
                dist_pos=dist_pos_e+230/4*3
            
            traci.vehicle.subscribe(testVname,(tc.VAR_LANE_INDEX,tc.VAR_SPEED,tc.VAR_POSITION,tc.VAR_LANEPOSITION))
        
            pos_e[j]=dist_pos
            
        pos_total=np.vstack((pos_total,pos_e))
            
        vehicle_obs_rl1=traci.vehicle.getSubscriptionResults("autonomous_vehicle")
        lane_pos=vehicle_obs_rl1[tc.VAR_LANEPOSITION]        
        vehID=traci.vehicle.getIDList()

# new_state and reward
        speed_total_e=[]
        done=1           
        if done==1 and len(vehID) == num_veh:
            reward=0
            vehicle_obs_rl1=traci.vehicle.getSubscriptionResults("autonomous_vehicle")
            
            lane_rl1=vehicle_obs_rl1[tc.VAR_LANE_INDEX]
            veh=vehicle_obs_rl1[tc.VAR_SPEED]
            veh_len=len(vehID)
            #set the line
            vehposition=vehicle_obs_rl1[tc.VAR_POSITION]
            vehpos_1=vehposition[0]-r
            vehpos_2=vehposition[1]-r
            slope=vehpos_2/vehpos_1

            theta_total=1000*np.ones(veh_len)
            theta_total_01=1000*np.ones(veh_len)
            speed_total_e=veh

            for i in range(1,len(vehID)):
                vehicle_obs_e=traci.vehicle.getSubscriptionResults(vehID[i])
                lane_e=vehicle_obs_e[tc.VAR_LANE_INDEX]
                veh_e=vehicle_obs_e[tc.VAR_SPEED]
                speed_total_e=np.hstack((speed_total_e,veh_e))

                vehposition_e=vehicle_obs_e[tc.VAR_POSITION]
                vehpos_1_e=vehposition_e[0]-r
                vehpos_2_e=vehposition_e[1]-r
                ind_e=vehpos_2_e-slope*vehpos_1_e
    
                if slope>0 and vehpos_1 !=0 and vehpos_1>0:
                    if ind_e<=0 and lane_e != lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total[i]=theta
                    if ind_e<=0 and lane_e == lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total_01[i]=theta
   
                if slope>0 and vehpos_1 !=0 and vehpos_1<0:
                    if ind_e>=0 and lane_e != lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total[i]=theta
                    if ind_e>=0 and lane_e == lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total_01[i]=theta
    
                if slope<0 and vehpos_1 !=0 and vehpos_1>0:
                    if ind_e<=0 and lane_e != lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total[i]=theta
                    if ind_e<=0 and lane_e == lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total_01[i]=theta
    
                if slope<0 and vehpos_1!=0 and vehpos_1<0:
                    if ind_e>=0 and lane_e != lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total[i]=theta
                    if ind_e>=0 and lane_e == lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total_01[i]=theta
    
                if slope==0 and vehpos_1>0:
                    if vehpos_2_e<=0 and lane_e != lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total[i]=theta
                    if vehpos_2_e<=0 and lane_e == lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total_01[i]=theta
                        
    
                if slope==0 and vehpos_1<0:
                    if vehpos_2_e>=0 and lane_e != lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total[i]=theta
                    if vehpos_2_e>=0 and lane_e == lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total_01[i]=theta
                        
    
                if vehpos_1==0 and vehpos_2>0:
                    if vehpos_1_e>=0 and lane_e != lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total[i]=theta
                    if vehpos_1_e>=0 and lane_e == lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total_01[i]=theta
    
                if vehpos_1==0 and vehpos_2<0:
                    if vehpos_1_e<=0 and lane_e != lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total[i]=theta
                    if vehpos_1_e<=0 and lane_e == lane_rl1:
                        cos=(vehpos_1*vehpos_1_e+vehpos_2*vehpos_2_e)/(((vehpos_1**2+vehpos_2**2)**0.5)*((vehpos_1_e**2+vehpos_2_e**2)**0.5))
                        theta=math.acos(cos)
                        theta_total_01[i]=theta


            y=np.where(theta_total==min(theta_total))
            veh_nextBehind=vehID[int(y[0])]
            vehicle_obs_e=traci.vehicle.getSubscriptionResults(veh_nextBehind)
            speed_nextBehind=vehicle_obs_e[tc.VAR_SPEED]

            y_01=np.where(theta_total_01==min(theta_total_01))
            veh_Behind=vehID[int(y_01[0])]
            vehicle_obs_e=traci.vehicle.getSubscriptionResults(veh_Behind)
            speed_Behind=vehicle_obs_e[tc.VAR_SPEED]
            dist_Behind=(min(theta_total_01))*r
            
            headway=traci.vehicle.getLeader(veh_nextBehind,200)
            headway1=headway[0]
            headway2=headway[1]
            
            vehicle_header_next=traci.vehicle.getSubscriptionResults(headway1)
            veh_header_next=vehicle_header_next[tc.VAR_SPEED]
            speed_nextLeader=veh_header_next
            
            vehicle_obs_e=traci.vehicle.getSubscriptionResults(veh_nextBehind)
            vehicle_obs_ee=traci.vehicle.getSubscriptionResults(headway1)

            vehposition_e=vehicle_obs_e[tc.VAR_POSITION]
            vehpos_1_e=vehposition_e[0]-r
            vehpos_2_e=vehposition_e[1]-r
            
            vehposition_ee=vehicle_obs_ee[tc.VAR_POSITION]
            vehpos_1_ee=vehposition_ee[0]-r
            vehpos_2_ee=vehposition_ee[1]-r
            cos_e=(vehpos_1_e*vehpos_1_ee+vehpos_2_e*vehpos_2_ee)/(((vehpos_1_e**2+vehpos_2_e**2)**0.5)*((vehpos_1_ee**2+vehpos_2_ee**2)**0.5))
            theta_e=math.acos(cos_e)
            dist=theta_e*r
            headway_nextBehind=dist

            headway=traci.vehicle.getLeader(rl1,200)
            headway1=headway[0]
            headway2=headway[1]            
            header_rl1=headway1

            vehicle_header=traci.vehicle.getSubscriptionResults(header_rl1)
            veh_header=vehicle_header[tc.VAR_SPEED]
            dist_header=headway2+7
            currentLaneIndex=lane_rl1
            
            if this_lane !=currentLaneIndex:
                last_lc=traci.simulation.getTime()


            diff_dist=(min(theta_total))*r #distance of AV and nextBehind
            
            new_state=np.array([veh,veh_header,dist_header,currentLaneIndex,speed_Behind,dist_Behind,speed_nextBehind,speed_nextLeader,headway_nextBehind,diff_dist])
            
        state = new_state
        speed_total=np.vstack((speed_total,speed_total_e))

    traci.close()




 Retrying in 1 seconds
autonomous_vehicle
(-1.414933198573102, 40.25168137455436)
54.02996040254613
169.02996040254612
54.02996040254613
(-1.414933198573102, 40.25168137455436)
v20_0
v19_1
v0_1
autonomous_vehicle
(-1.417980769947784, 40.2205961465326)
54.05991801804118
169.05991801804117
54.05991801804118
(-1.417980769947784, 40.2205961465326)
v20_0
v19_1
v0_1
autonomous_vehicle
(-1.4220438927082983, 40.179152294375356)
54.09985849995405
169.09985849995405
54.09985849995405
(-1.4220438927082983, 40.179152294375356)
v20_0
v19_1
v0_1
autonomous_vehicle
(-1.4271223894999079, 40.127351627100936)
54.14978010488833
169.14978010488832
54.14978010488833
(-1.4271223894999079, 40.127351627100936)
v20_0
v19_1
v0_1
autonomous_vehicle
(-1.433216065761123, 40.06519612923654)
54.209680920305274
169.2096809203053
54.209680920305274
(-1.433216065761123, 40.06519612923654)
v20_0
v19_1
v0_1
autonomous_vehicle
(-1.4403247077934735, 39.99268798050657)
54.27955884554971
169.2795588455497
54.27955884554971
(

8.615788601951568
(-0.5314253431356701, 27.681863623242464)
v20_0
v19_1
v0_1
autonomous_vehicle
(-0.40859768786209993, 27.18903737957925)
9.102984724011195
181.6029847240112
9.102984724011195
(-0.40859768786209993, 27.18903737957925)
v20_0
v19_1
v0_1
autonomous_vehicle
(-0.27679408128702887, 26.688859590525137)
9.599095301891154
182.09909530189117
9.599095301891154
(-0.27679408128702887, 26.688859590525137)
v20_0
v19_1
v0_1
autonomous_vehicle
(-0.14264359299629997, 26.17977568624237)
10.104039574648102
182.6040395746481
10.104039574648102
(-0.14264359299629997, 26.17977568624237)
v20_0
v19_1
v0_1
autonomous_vehicle
(0.0088627869515443, 25.66621811097988)
10.617731705564681
183.1177317055647
10.617731705564681
(0.0088627869515443, 25.66621811097988)
v20_0
v19_1
v0_1
autonomous_vehicle
(0.17252098983442457, 25.146781206177696)
11.140080534853704
183.6400805348537
11.140080534853704
(0.17252098983442457, 25.146781206177696)
v20_0
v19_1
v0_1
autonomous_vehicle
(0.33886113227761483, 24.6188

v20_0
v19_1
v0_1
autonomous_vehicle
(23.473706218077076, 0.7380067784694422)
44.6658325163606
217.1658325163606
44.6658325163606
(23.473706218077076, 0.7380067784694422)
v20_0
v19_1
v0_1
autonomous_vehicle
(24.11370315642057, 0.512904407052075)
45.31653279705427
217.81653279705426
45.31653279705427
(24.11370315642057, 0.512904407052075)
v20_0
v19_1
v0_1
autonomous_vehicle
(24.749352051445566, 0.29773839475002706)
45.96026235226035
218.46026235226034
45.96026235226035
(24.749352051445566, 0.29773839475002706)
v20_0
v19_1
v0_1
autonomous_vehicle
(25.382208666053657, 0.09834521480501224)
46.596666713776095
219.0966667137761
46.596666713776095
(25.382208666053657, 0.09834521480501224)
v20_0
v19_1
v0_1
autonomous_vehicle
(26.007819994436876, -0.09733094447998701)
47.225402996577024
219.72540299657703
47.225402996577024
(26.007819994436876, -0.09733094447998701)
v20_0
v19_1
v0_1
autonomous_vehicle
(26.633646561583404, -0.2622447020388695)
47.84614063562101
220.346140635621
47.84614063562101


autonomous_vehicle
(44.3942085189326, -0.796574184353371)
7.48952921005993
7.48952921005993
7.48952921005993
(44.3942085189326, -0.796574184353371)
v20_0
v19_1
v0_1
autonomous_vehicle
(44.56877662703014, -0.7561113778406954)
7.661401037400026
7.661401037400026
7.661401037400026
(44.56877662703014, -0.7561113778406954)
v20_0
v19_1
v0_1
autonomous_vehicle
(44.7366689846896, -0.7171959307010851)
7.8267002241864505
7.8267002241864505
7.8267002241864505
(44.7366689846896, -0.7171959307010851)
v20_0
v19_1
v0_1
autonomous_vehicle
(44.89820318953384, -0.6797542275914941)
7.985739462607908
7.985739462607908
7.985739462607908
(44.89820318953384, -0.6797542275914941)
v20_0
v19_1
v0_1
autonomous_vehicle
(45.05369901634983, -0.643712148528185)
8.138833588389979
8.138833588389979
8.138833588389979
(45.05369901634983, -0.643712148528185)
v20_0
v19_1
v0_1
autonomous_vehicle
(45.20347746612984, -0.6089952893076521)
8.286298644524248
8.286298644524248
8.286298644524248
(45.20347746612984, -0.60899528930

12.857531825534107
(49.76927380471174, 0.7460894071744724)
v19_0
v19_1
v0_1
autonomous_vehicle
(49.8922353923447, 0.7893379655833084)
12.982549829205192
12.982549829205192
12.982549829205192
(49.8922353923447, 0.7893379655833084)
v19_0
v19_1
v0_1
autonomous_vehicle
(50.01710132313136, 0.8332563274462006)
13.109504024364707
13.109504024364707
13.109504024364707
(50.01710132313136, 0.8332563274462006)
v19_0
v19_1
v0_1
autonomous_vehicle
(50.143867497992424, 0.877843051018026)
13.238390243380165
13.238390243380165
13.238390243380165
(50.143867497992424, 0.877843051018026)
v19_0
v19_1
v0_1
autonomous_vehicle
(50.270475956570706, 0.928525038063203)
13.369197872313231
13.369197872313231
13.369197872313231
(50.270475956570706, 0.928525038063203)
v19_0
v19_1
v0_1
autonomous_vehicle
(50.39882379259558, 0.9802206942398862)
13.501909990245043
13.501909990245043
13.501909990245043
(50.39882379259558, 0.9802206942398862)
v19_0
v19_1
v0_1
autonomous_vehicle
(50.52899116760033, 1.0326492202834676)
13

...

autonomous_vehicle
(14.013762866881011, 5.812460632910624)
34.33447601614698
206.834476016147
34.33447601614698
(14.013762866881011, 5.812460632910624)
v20_0
v19_1
v0_1
autonomous_vehicle
(14.411929224613168, 5.522595524481614)
34.806847864229205
207.3068478642292
34.806847864229205
(14.411929224613168, 5.522595524481614)
v20_0
v19_1
v0_1
autonomous_vehicle
(14.815096352940168, 5.2290898550595575)
35.285152466804234
207.78515246680422
35.285152466804234
(14.815096352940168, 5.2290898550595575)
v20_0
v19_1
v0_1
autonomous_vehicle
(15.234712882860467, 4.948559053021091)
35.76939171561327
208.26939171561327
35.76939171561327
(15.234712882860467, 4.948559053021091)
v20_0
v19_1
v0_1
autonomous_vehicle
(15.662983421040968, 4.669685214205882)
36.259566733422496
208.7595667334225
36.259566733422496
(15.662983421040968, 4.669685214205882)
v20_0
v19_1
v0_1
autonomous_vehicle
(16.096440390134294, 4.387434164563715)
36.755677856189735
209.25567785618972
36.755677856189735
(16.096440390134294, 4.38

autonomous_vehicle
(42.84883405381503, -1.0805396062714847)
5.982337648132166
5.982337648132166
5.982337648132166
(42.84883405381503, -1.0805396062714847)
v20_0
v19_1
v0_1
autonomous_vehicle
(43.598314329558505, -0.9465265768339094)
6.712585392726605
6.712585392726605
6.712585392726605
(43.598314329558505, -0.9465265768339094)
v20_0
v19_1
v0_1
autonomous_vehicle
(44.34981810215156, -0.8068633538059296)
7.445824422675271
7.445824422675271
7.445824422675271
(44.34981810215156, -0.8068633538059296)
v20_0
v19_1
v0_1
autonomous_vehicle
(45.0972990150669, -0.633606188560652)
8.18176016606327
8.18176016606327
8.18176016606327
(45.0972990150669, -0.633606188560652)
v20_0
v19_1
v0_1
autonomous_vehicle
(45.84655048382409, -0.4571927779112201)
8.92007214041155
8.92007214041155
8.92007214041155
(45.84655048382409, -0.4571927779112201)
v20_0
v19_1
v0_1
autonomous_vehicle
(46.592960422014556, -0.2605036725772447)
9.66041262308531
9.66041262308531
9.66041262308531
(46.592960422014556, -0.260503672577

37.93049696597419
(69.47398491647512, 17.139900300727998)
v20_0
v19_1
v0_1
autonomous_vehicle
(69.5856209499764, 17.322704305586345)
38.13593803680651
38.13593803680651
38.13593803680651
(69.5856209499764, 17.322704305586345)
v20_0
v19_1
v0_1
autonomous_vehicle
(69.68999019504555, 17.493608944387084)
38.32800621404715
38.32800621404715
38.32800621404715
(69.68999019504555, 17.493608944387084)
v20_0
v19_1
v0_1
autonomous_vehicle
(69.78052207844138, 17.656898386345752)
38.507099899106066
38.507099899106066
38.507099899106066
(69.78052207844138, 17.656898386345752)
v20_0
v19_1
v0_1
autonomous_vehicle
(69.86207729082518, 17.805681544072982)
38.669834288442964
38.669834288442964
38.669834288442964
(69.86207729082518, 17.805681544072982)
v20_0
v19_1
v0_1
autonomous_vehicle
(69.93612259210877, 17.940764188306552)
38.81758348233806
38.81758348233806
38.81758348233806
(69.93612259210877, 17.940764188306552)
v20_0
v19_1
v0_1
autonomous_vehicle
(70.00324576683956, 18.06321862869379)
38.9515203397

v0_1
autonomous_vehicle
(70.39033288748928, 18.769391078527754)
39.723909796607444
39.723909796607444
39.723909796607444
(70.39033288748928, 18.769391078527754)
v19_0
v19_1
v0_1
autonomous_vehicle
(70.39504977996701, 18.77799622021009)
39.73332183287703
39.73332183287703
39.73332183287703
(70.39504977996701, 18.77799622021009)
v19_0
v19_1
v0_1
autonomous_vehicle
(70.40036109058016, 18.787685773355697)
39.74391996466954
39.74391996466954
39.74391996466954
(70.40036109058016, 18.787685773355697)
v19_0
v19_1
v0_1
autonomous_vehicle
(70.40630523051149, 18.798529812419613)
39.75578083724388
39.75578083724388
39.75578083724388
(70.40630523051149, 18.798529812419613)
v19_0
v19_1
v0_1
autonomous_vehicle
(70.41292095757684, 18.81059904422803)
39.768981787526904
39.768981787526904
39.768981787526904
(70.41292095757684, 18.81059904422803)
v19_0
v19_1
v0_1
autonomous_vehicle
(70.42024733716016, 18.823964736711105)
39.78360076616349
39.78360076616349
39.78360076616349
(70.42024733716016, 18.8239647

autonomous_vehicle
(72.24050201061794, 22.82297050912042)
43.99994532409695
43.99994532409695
43.99994532409695
(72.24050201061794, 22.82297050912042)
v19_0
v19_1
v0_1
autonomous_vehicle
(72.31204228588788, 23.000587744273346)
44.18360217557491
44.18360217557491
44.18360217557491
(72.31204228588788, 23.000587744273346)
v19_0
v19_1
v0_1
autonomous_vehicle
(72.38038218840593, 23.18481210429138)
44.37210692227002
44.37210692227002
44.37210692227002
(72.38038218840593, 23.18481210429138)
v20_0
v19_1
v0_1
autonomous_vehicle
(72.4472850412546, 23.37502609768464)
44.56550206446119
44.56550206446119
44.56550206446119
(72.4472850412546, 23.37502609768464)
v20_0
v19_1
v0_1
autonomous_vehicle
(72.51589407847193, 23.5700910074202)
44.7638292499739
44.7638292499739
44.7638292499739
(72.51589407847193, 23.5700910074202)
v20_0
v19_1
v0_1
autonomous_vehicle
(72.58622341677923, 23.770046969274297)
44.9671292858172
44.9671292858172
44.9671292858172
(72.58622341677923, 23.770046969274297)
v20_0
v19_1
v0_

In [11]:
import scipy.io as sio
import matplotlib.pyplot as plt
sio.savemat('saveddata_230_speed_pos_nocontrol.mat', {'speed_230_new': speed_total,'pos_230_new':pos_total})
